# Importing Python Packages

In [66]:
import openpyxl
import xlrd
import os
import glob

from pandas import DataFrame as df
from scipy.stats import trim_mean, kurtosis
from scipy.stats.mstats import mode, gmean, hmean

In [67]:
# import python packages

# import nltk (natural language tool kit)
import nltk
# stopwords, FreqDist, word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist, word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import requests
from lxml import html
import csv
import pandas as pd

from os import path

import matplotlib.pyplot as plt
%matplotlib inline


#regular expression
import re
#csv
import csv
# Excel-like format
import pandas as pd
#data visualization
import matplotlib.pyplot as plt
% matplotlib inline

import numpy as np

import statsmodels.api as sm

import statsmodels.formula.api as smf

# Importing Data

## Opening the Correct Directory

In [68]:
os.chdir('C:\\Users\\millsj.ADMIN\\Desktop\\GENBA 894\\2018_DataChallenges_Teradata\\Bike Teams\\')

In [69]:
# filenames
excel_names = ["2013-2017 Bike teams.xlsx"]

# read them in
excels = [pd.ExcelFile(name) for name in excel_names]

# turn them into dataframes
frames = [x.parse(x.sheet_names[0],index_col=None) for x in excels]

# delete the first row for all frames except the first
# i.e. remove the header row -- assumes it's the first
frames[1:] = [df[1:] for df in frames[1:]]

# concatenate them..
biketeams = pd.concat(frames)

# Check headers
biketeams.head(3)

,Event Type,Internal Event Name,Event ID,Team ID,Team Name,Team Creation Date,Team Captain Contact ID,Captain Email Domain,Team Captain Accept Email,Team Division,...,Total Offline Unconfirmed Gifts($),Team Goal($),Total Confirmed Gifts in Team History($),Previous Event Fiscal Year,Previous Event Internal Name,Previous Event Team Name,Previous Event Confirmed Gifts($),Previous Event Team Members,Event Date,Fiscal Year
0,Bike,2013 Bike MS Hawaii,21136,320012,AKAMAI HAWAII,2013-01-13 00:38:00,9560000.0,gmail.com,1.0,Ohana,...,575.0,1500.0,4551.0,NaN,NaN,NaN,0.0,0,2013-08-03 15:57:00,2013.0
1,Bike,2013 Bike MS Hawaii,21136,344048,Are We There Yet?,2013-04-16 21:02:00,11751773.0,gmail.com,1.0,Ohana,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0,2013-08-03 15:57:00,2013.0
2,Bike,2013 Bike MS Hawaii,21136,351448,Big Island Honda,2013-07-29 16:47:00,12450037.0,fjhawaii.com,1.0,Corporate,...,0.0,0.0,35.0,NaN,NaN,NaN,0.0,0,2013-08-03 15:57:00,2013.0


In [70]:
# convert string to integers
#biketeams['Team Captain Accept Email'] = (biketeams['Team Captain Accept Email'] == 'TRUE').astype(int)

In [71]:
# make sure the conversion worked
#biketeams.head(3)

# Looking for Data Anomalies

In [72]:
# List of all headers
# biketeams['Team Creation Date','Team Captain Contact ID','Captain Email Domain','Team Captain Accept Email','Team Division','Company','Number of Participants','Total Fees Paid','Team Total Confirmed ($)','Total Online Gifts($)','Total Offline Confirmed Gifts($)','Total Offline Unconfirmed Gifts($)','Team Goal($)','Total Confirmed Gifts in Team History($)','Previous Event Fiscal Year','Previous Event Internal Name','Previous Event Team Name','Previous Event Confirmed Gifts($)','Previous Event Team Members','Event Date','Fiscal Year']

In [73]:
# counting zero values
print((biketeams[['Team Creation Date','Team Captain Contact ID','Captain Email Domain','Team Captain Accept Email','Team Division','Company','Number of Participants','Total Fees Paid','Team Total Confirmed ($)','Total Online Gifts($)','Total Offline Confirmed Gifts($)','Total Offline Unconfirmed Gifts($)','Team Goal($)','Total Confirmed Gifts in Team History($)','Previous Event Fiscal Year','Previous Event Internal Name','Previous Event Team Name','Previous Event Confirmed Gifts($)','Previous Event Team Members','Event Date','Fiscal Year']] == 0).sum())



Team Creation Date                              0
Team Captain Contact ID                         0
Captain Email Domain                            0
Team Captain Accept Email                    8056
Team Division                                   0
Company                                         0
Number of Participants                         21
Total Fees Paid                              3320
Team Total Confirmed ($)                     3197
Total Online Gifts($)                        4197
Total Offline Confirmed Gifts($)             8783
Total Offline Unconfirmed Gifts($)          26245
Team Goal($)                                 8452
Total Confirmed Gifts in Team History($)     2778
Previous Event Fiscal Year                      0
Previous Event Internal Name                    0
Previous Event Team Name                        1
Previous Event Confirmed Gifts($)           24855
Previous Event Team Members                 24638
Event Date                                      0


In [74]:
# Counting missing values
biketeamsnull=(biketeams[['Team Creation Date','Team Captain Contact ID','Captain Email Domain','Team Captain Accept Email','Team Division','Company','Number of Participants','Total Fees Paid','Team Total Confirmed ($)','Total Online Gifts($)','Total Offline Confirmed Gifts($)','Total Offline Unconfirmed Gifts($)','Team Goal($)','Total Confirmed Gifts in Team History($)','Previous Event Fiscal Year','Previous Event Internal Name','Previous Event Team Name','Previous Event Confirmed Gifts($)','Previous Event Team Members','Event Date','Fiscal Year']])

print(biketeamsnull.isnull().sum())


Team Creation Date                              0
Team Captain Contact ID                       552
Captain Email Domain                          698
Team Captain Accept Email                     552
Team Division                                2662
Company                                     23166
Number of Participants                          0
Total Fees Paid                                 0
Team Total Confirmed ($)                        0
Total Online Gifts($)                           0
Total Offline Confirmed Gifts($)                0
Total Offline Unconfirmed Gifts($)              0
Team Goal($)                                    0
Total Confirmed Gifts in Team History($)        1
Previous Event Fiscal Year                  24632
Previous Event Internal Name                24632
Previous Event Team Name                    24631
Previous Event Confirmed Gifts($)               0
Previous Event Team Members                     0
Event Date                                      0


In [75]:
#compute count of values in columns
biketeams['Captain Email Domain'].value_counts()

gmail.com              10915
yahoo.com               3577
hotmail.com             1808
aol.com                 1513
comcast.net             1327
verizon.net              537
msn.com                  523
sbcglobal.net            378
me.com                   243
cox.net                  242
att.net                  234
mac.com                  191
charter.net              152
optonline.net            148
bellsouth.net            145
live.com                 137
nmss.org                 136
outlook.com              128
ymail.com                 97
icloud.com                87
earthlink.net             66
deloitte.com              62
roadrunner.com            56
q.com                     55
embarqmail.com            50
rocketmail.com            43
rochester.rr.com          35
wellsfargo.com            32
suddenlink.net            32
kpmg.com                  31
                       ...  
badcatflims.com            1
scotberkey.com             1
bushwick.com               1
omahatelevisio

In [76]:


#compute count of values in columns
biketeams['Team Captain Accept Email'].value_counts()

# originally found the following results: "True 24523, False 8056, Friends and Family 1" which identified and data entry error and has been corrected.

1.0    24524
0.0     8056
Name: Team Captain Accept Email, dtype: int64

# Describing Data

In [77]:
biketeams.describe()

,Event ID,Team ID,Team Captain Contact ID,Team Captain Accept Email,Number of Participants,Total Fees Paid,Total Online Gifts($),Total Offline Confirmed Gifts($),Total Offline Unconfirmed Gifts($),Team Goal($),Total Confirmed Gifts in Team History($),Previous Event Fiscal Year,Previous Event Confirmed Gifts($),Fiscal Year
count,33132.000000,33132.000000,3.258000e+04,32580.000000,33132.000000,33132.000000,33132.000000,33132.000000,33132.000000,3.313200e+04,3.313100e+04,8500.000000,3.313200e+04,33131.000000
mean,24480.767777,411722.021248,8.953652e+06,0.752732,10.701074,440.104223,6320.985665,2682.467439,101.388737,6.945186e+04,1.665733e+04,2014.977882,3.677016e+03,2014.938456
std,3081.189125,67696.166584,4.614824e+06,0.431431,21.691457,1512.024957,19529.446091,10539.766637,791.310748,7.769344e+06,6.292635e+04,1.220864,2.087463e+04,1.419003
min,17966.000000,266344.000000,1.566043e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,2011.000000,0.000000e+00,2013.000000
25%,22247.000000,353140.500000,4.813076e+06,1.000000,2.000000,55.000000,405.000000,0.000000,0.000000,0.000000e+00,8.850000e+02,2015.000000,0.000000e+00,2014.000000
50%,25096.000000,405873.500000,9.622729e+06,1.000000,5.000000,140.000000,1600.000000,350.000000,0.000000,2.000000e+03,3.065000e+03,2015.000000,0.000000e+00,2015.000000
75%,27091.000000,466838.250000,1.280501e+07,1.000000,10.000000,340.000000,4920.962500,1571.000000,0.000000,5.000000e+03,1.066500e+04,2016.000000,0.000000e+00,2016.000000
max,29050.000000,538680.000000,1.636517e+07,1.000000,725.000000,71200.000000,751129.190000,498352.720000,39638.000000,1.000000e+09,2.650325e+06,2016.000000,1.249482e+06,2018.000000


In [78]:
# counting 1 values
print((biketeams[['Team Creation Date','Team Captain Contact ID','Captain Email Domain','Team Captain Accept Email','Team Division','Company','Number of Participants','Total Fees Paid','Team Total Confirmed ($)','Total Online Gifts($)','Total Offline Confirmed Gifts($)','Total Offline Unconfirmed Gifts($)','Team Goal($)','Total Confirmed Gifts in Team History($)','Previous Event Fiscal Year','Previous Event Internal Name','Previous Event Team Name','Previous Event Confirmed Gifts($)','Previous Event Team Members','Event Date','Fiscal Year']] == 1).sum())



Team Creation Date                              0
Team Captain Contact ID                         0
Captain Email Domain                            0
Team Captain Accept Email                   24524
Team Division                                   0
Company                                         0
Number of Participants                       4515
Total Fees Paid                                 1
Team Total Confirmed ($)                        4
Total Online Gifts($)                           2
Total Offline Confirmed Gifts($)                5
Total Offline Unconfirmed Gifts($)              6
Team Goal($)                                    5
Total Confirmed Gifts in Team History($)        4
Previous Event Fiscal Year                      0
Previous Event Internal Name                    0
Previous Event Team Name                        0
Previous Event Confirmed Gifts($)               1
Previous Event Team Members                   534
Event Date                                      0


# FIRST PRIORITY: CORPORATE ACQUISITION
  * What are the greatest growth opportunities for new corporate teams?
  * Can we apply those opportunities to specific rides/markets, especially our biggest events?
  * What industries have had the strongest involvement in Bike MS in the last five years?
  * What occupations were responsible for most of our fundraising?
  * Can we tie together these industries and occupations to identify gaps/opportunities?
  * What is the common denominator for our top performing corporate teams? (Is it industry, culture, executive involvement, connection to MS, other?)
  * Can we quantify the effect competing events are having in our top markets? 
Top competitors:
   1. Best Buddies
     * HYANNIS PORT June 2, 2018 Boston to Hyannis Port, MA
     * HEARST CASTLE September 8, 2018 San Simeon, CA
     * MIAMI November 16, 2018 Miami, FL
   2. Peddle the Cause
     * Louis
     * San Diego
   3. Pelatonia – OH
   4. PanMass Challenge - Boston
   5. ADA – Tour de Cure
   6. Jude Children’s Research Hospital
   7. JDRF Ride to Cure Diabetes
   8. Cycle for Life - Cystic Fibrosis Foundation
   9. Ride to Recovery
   10. Virtual Cycling
   11. Athletic Club Stationary Cycling Relays
   12. American Cancer Society
   13. Leukemia and Lymphoma Society – Team in Training

# SECOND PRIORITY: DIGITAL/SOCIAL ACQUISITION
  * What are the greatest opportunities for digital marketing investments? Where have we seen the greatest ROI?
  * Once someone is registered, what tactics and behaviors drive fundraising, and at what times leading up to the event?
  * What behavioral data do you see about usage of our fundraising tools and how it may or may not relate to performance of top fundraisers?
  * Despite increasing our digital advertising spend, acquisition continues to trend downward overall. Why? Is it an issue of needing more traffic, better targeting, or a conversion rate issue that needs to be addressed through the registration process? What can we do to reverse the trend?


# FINALLY:
  * As you studied this data, is there something else that came up as an insight into our operations that the questions above do not capture? 


# Descriptive Analytics